<a href="https://colab.research.google.com/github/ShinyRyo/TPU/blob/master/Copy_of_TPU_DCGAN_CelebA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow2.0+Colab TPUによるDCGANのサンプル
作成者：こしあん（koshian2）

## CelebAのダウンロード
コードは　https://gist.github.com/charlesreid1/4f3d676b33b95fce83af08e4ec261822　より


In [0]:
import requests
 
def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value
 
        return None
 
    def save_response_content(response, destination):
        CHUNK_SIZE = 32768
 
        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
 
    URL = "https://docs.google.com/uc?export=download"
 
    session = requests.Session()
 
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
 
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
 
    save_response_content(response, destination)

In [0]:
download_file_from_google_drive("0B7EVK8r0v71pZjFTYXZWM3FlRnM", "celebA.zip")

In [0]:
!unzip celebA.zip > /dev/null

CelebAのファイル数

In [4]:
!ls img_align_celeba -U1 | wc -l

202599


## TF2.0にアップデート

In [0]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 72kB/s 
     |████████████████████████████████| 3.8MB 46.2MB/s 
     |████████████████████████████████| 450kB 32.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=9f6853b3608f5470da2d6d68a0b165727a020602fa0149faadd893543f6d79ad
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: t

## TPU用の初期設定
Strategyの初期化

In [6]:
import tensorflow as tf
import os
# tpu用
tpu_grpc_url = "grpc://" + os.environ["COLAB_TPU_ADDR"]
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
tf.config.experimental_connect_to_cluster(tpu_cluster_resolver) # TF2.0の場合、ここを追加
tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver) # TF2.0の場合、今後experimentialが取れる可能性がある    
strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)  # ここも同様

INFO:tensorflow:Initializing the TPU system: grpc://10.84.63.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.84.63.154:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


分散訓練用のデコレーター　詳細：https://blog.shikoan.com/distributed-train-decorator-in-tf20/

In [0]:
from enum import Enum

# Distribute trainingを楽にするためのデコレーター
class Reduction(Enum):
    NONE = 0
    SUM = 1
    MEAN = 2
    CONCAT = 3

def distrtibuted(*reduction_flags):
    def _decorator(fun):
        def per_replica_reduction(z, flag):
            if flag == Reduction.NONE:
                return z
            elif flag == Reduction.SUM:
                return strategy.reduce(tf.distribute.ReduceOp.SUM, z, axis=None)
            elif flag == Reduction.MEAN:
                return strategy.reduce(tf.distribute.ReduceOp.MEAN, z, axis=None)
            elif flag == Reduction.CONCAT:
                z_list = strategy.experimental_local_results(z)
                return tf.concat(z_list, axis=0)
            else:
                raise NotImplementedError()

        @tf.function
        def _decorated_fun(*args, **kwargs):
            fun_result = strategy.experimental_run_v2(fun, args=args, kwargs=kwargs)
            if len(reduction_flags) == 0:
                assert fun_result is None
                return
            elif len(reduction_flags) == 1:
                assert type(fun_result) is not tuple and fun_redult is not None
                return per_replica_reduction(fun_result, *reduction_flags)
            else:
                assert type(fun_result) is tuple
                return tuple((per_replica_reduction(fr, rf) for fr, rf in zip(fun_result, reduction_flags)))
        return _decorated_fun
    return _decorator

## メイン部分（DCGAN）
ロードがポイントで、現状のTPUではファイルを逐次読み込めないためNumpy配列に変換する。ただし、uint8のNumpy配列にしても、CelebAでは2GBの壁が問題になる（TensorFlowのオブジェクトは1個あたり2GBの制限がある）。

そのため、データを前後で分割し、2GB未満になるようにする。前をX1、後をX2として、(X1, X2)というデータセットを作り、個々のTPUデバイスの中でX1とX2を結合して訓練する。

DCGANの設定は[論文](https://arxiv.org/abs/1511.06434)の通り。バッチサイズ1024で訓練可能。

In [0]:
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
from PIL import Image
import glob
import pickle
import matplotlib.pyplot as plt

def load_celeba(batch_size):
    # list_filesがTPUで使えないので一旦メモリにNumpy配列にキャッシュする
    if not os.path.exists("image_cache.pkl"):
        image_paths = sorted(glob.glob("./img_align_celeba/*.jpg"))
        images = np.zeros((len(image_paths), 64, 64, 3), np.uint8)
        print("Create image caches...")
        for i, path in tqdm(enumerate(image_paths)):
            with Image.open(path) as img:
                img = img.resize((64, 64), Image.LANCZOS)
                images[i] = np.asarray(img, np.uint8)
        with open("image_cache.pkl", "wb") as fp:
            pickle.dump(images, fp)
    else:
        with open("image_cache.pkl", "rb") as fp:
            images = pickle.load(fp)
    
    # さらにテンソルが2GBまでしか入らないため前後でデータを分割する（ちょっと頭おかしい）
    split_n = images.shape[0] // 2
    images1, images2 = images[:split_n], images[split_n:2 * split_n]
    del images

    def preprocess(img):
        x = tf.cast(img, tf.float32) / 127.5 - 1.0  # [-1. 1]
        return x
    
    dataset = tf.data.Dataset.from_tensor_slices((images1, images2))
    dataset = dataset.map(
        lambda x1, x2: (preprocess(x1), preprocess(x2))
    ).shuffle(4096).batch(batch_size//2).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_initializers():
    return (tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02), # conv initializer
            tf.keras.initializers.RandomNormal(mean=1.0, stddev=0.02)) # bn gamma initializer

def create_generator(ngf=128):
    conv_initializer, bn_gamma_initializer = get_initializers()
    
    inputs = layers.Input((1, 1, 100))
    for i, k in enumerate([8, 4, 2, 1]):
        if i == 0:
            x = layers.Conv2DTranspose(ngf * k, 4, strides=1, padding="valid", use_bias=False,
                                       kernel_initializer=conv_initializer)(inputs)
        else:
            x = layers.Conv2DTranspose(ngf * k, 4, strides=2, padding="same", use_bias=False,
                                       kernel_initializer=conv_initializer)(x)
        x = layers.BatchNormalization(gamma_initializer=bn_gamma_initializer)(x)
        x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(3, 4, strides=2, padding="same", use_bias=False,
                               kernel_initializer=conv_initializer)(x)
    x = layers.Activation("tanh")(x)
    return tf.keras.models.Model(inputs, x)

def create_discriminator(ndf=128):
    conv_initializer, bn_gamma_initializer = get_initializers()

    inputs = layers.Input((64, 64, 3))
    x = inputs
    for i, k in enumerate([1, 2, 4, 8]):
        x = layers.Conv2D(ndf * k, 4, strides=2, padding="same", use_bias=False,        
                          kernel_initializer=conv_initializer)(x)                          
        if i != 0:
            x = layers.BatchNormalization(gamma_initializer=bn_gamma_initializer)(x)
        x = layers.LeakyReLU(0.2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.models.Model(inputs, x)

### save image utils
def make_grid(imgs, nrow, padding=0):
    assert imgs.ndim == 4 and nrow > 0
    batch, height, width, ch = imgs.shape
    n = nrow * (batch // nrow + np.sign(batch % nrow))
    ncol = n // nrow
    pad = np.zeros((n - batch, height, width, ch), imgs.dtype)
    x = np.concatenate([imgs, pad], axis=0)
    # border padding if required
    if padding > 0:
        x = np.pad(x, ((0, 0), (0, padding), (0, padding), (0, 0)),
                   "constant", constant_values=(0, 0))
        height += padding
        width += padding
    x = x.reshape(ncol, nrow, height, width, ch)
    x = x.transpose([0, 2, 1, 3, 4])  # (ncol, height, nrow, width, ch)
    x = x.reshape(height * ncol, width * nrow, ch)
    if padding > 0:
        x = x[:(height * ncol - padding),:(width * nrow - padding),:]
    return x

def save_img(imgs, filepath, nrow, padding=0):
    grid_img = make_grid(imgs, nrow, padding=padding)
    grid_img = ((grid_img + 1.0) * 127.5).astype(np.uint8)
    with Image.fromarray(grid_img) as img:
        img.save(filepath)
###


def main():
    batch_size = 1024
    dataset = load_celeba(batch_size)
    out_dir = "celeba_out"

    with strategy.scope():
        model_G = create_generator()
        model_D = create_discriminator()
        param_G = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
        param_D = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

        dataset = strategy.experimental_distribute_dataset(dataset)
        loss_func = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

        @distrtibuted(Reduction.SUM, Reduction.SUM, Reduction.CONCAT)
        def train_on_batch(real_img1, real_img2):
            # concat x1, x2
            real_img = tf.concat([real_img1, real_img2], axis=0)
            # generate fake images
            with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:
                z = tf.random.normal(shape=(real_img.shape[0], 1, 1, 100))
                fake_img = model_G(z)
            # train discriminator
            with d_tape:
                fake_out = model_D(fake_img)
                real_out = model_D(real_img)
                d_loss = (loss_func(tf.zeros(shape=(z.shape[0], 1)), fake_out)
                            + loss_func(tf.ones(shape=(z.shape[0], 1)), real_out)) / 2.0
                d_loss = tf.reduce_sum(d_loss) * (1.0 / batch_size)
            gradients = d_tape.gradient(d_loss, model_D.trainable_weights)
            param_D.apply_gradients(zip(gradients, model_D.trainable_weights))
            # train generator
            with g_tape:
                fake_out = model_D(fake_img)
                g_loss = loss_func(tf.ones(shape=(z.shape[0], 1)), fake_out)
                g_loss = tf.reduce_sum(g_loss) * (1.0 / batch_size)
            gradients = g_tape.gradient(g_loss, model_G.trainable_weights)
            param_G.apply_gradients(zip(gradients, model_G.trainable_weights))
            return d_loss, g_loss, fake_img 

        for epoch in range(100): # 100 epoch訓練
            with tqdm(dataset) as pbar:
                pbar.set_description(f"[Epoch {epoch}]")
                for step, (X1, X2) in enumerate(pbar):
                    d_loss, g_loss, fake = train_on_batch(X1, X2)
                    pbar.set_postfix({"g_loss": g_loss.numpy(), "d_loss": d_loss.numpy()})
                    if step == 200000 // batch_size - 1:
                        fake_img = fake # 末尾だと端数が出るため

            # save output
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            file_path = out_dir+f"/epoch_{epoch:04}.png"
            save_img(fake_img.numpy()[:64], file_path, 8)
            
            # 3エポックごとに画像表示
            if epoch % 3 == 0:
                with Image.open(file_path) as img:
                    plt.imshow(np.asarray(img))
                    plt.show()

In [25]:
main()

[Epoch 0]: : 197it [00:44,  4.40it/s, g_loss=5.13, d_loss=0.123]


TypeError: ignored

#確認

In [8]:
with tqdm(dataset) as pbar:
        pbar.set_description(f"[Epoch {epoch}]")

NameError: ignored

In [0]:
batch_size = 1024
dataset = load_celeba(batch_size)
out_dir = "celeba_out"

In [10]:
dataset

<PrefetchDataset shapes: ((None, 64, 64, 3), (None, 64, 64, 3)), types: (tf.float32, tf.float32)>

In [0]:
for epoch in range(100): # 100 epoch訓練
    with tqdm(dataset) as pbar:
        pbar.set_description(f"[Epoch {epoch}]")
        #print(epoch)

In [0]:
with strategy.scope():
    model_G = create_generator()
    model_D = create_discriminator()
    param_G = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    param_D = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

    dataset = strategy.experimental_distribute_dataset(dataset)
    loss_func = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

In [0]:
def train_on_batch(real_img1, real_img2):
    # concat x1, x2
    real_img = tf.concat([real_img1, real_img2], axis=0)
    # generate fake images
    with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:
        z = tf.random.normal(shape=(real_img.shape[0], 1, 1, 100))
        fake_img = model_G(z)
    # train discriminator
    with d_tape:
        fake_out = model_D(fake_img)
        real_out = model_D(real_img)
        d_loss = (loss_func(tf.zeros(shape=(z.shape[0], 1)), fake_out)
                    + loss_func(tf.ones(shape=(z.shape[0], 1)), real_out)) / 2.0
        d_loss = tf.reduce_sum(d_loss) * (1.0 / batch_size)
    gradients = d_tape.gradient(d_loss, model_D.trainable_weights)
    param_D.apply_gradients(zip(gradients, model_D.trainable_weights))
    # train generator
    with g_tape:
        fake_out = model_D(fake_img)
        g_loss = loss_func(tf.ones(shape=(z.shape[0], 1)), fake_out)
        g_loss = tf.reduce_sum(g_loss) * (1.0 / batch_size)
    gradients = g_tape.gradient(g_loss, model_G.trainable_weights)
    param_G.apply_gradients(zip(gradients, model_G.trainable_weights))
    return d_loss, g_loss, fake_img

In [26]:
#@distrtibuted(Reduction.SUM, Reduction.SUM, Reduction.CONCAT)
for epoch in range(100): # 100 epoch訓練
    with tqdm(dataset) as pbar:
        pbar.set_description(f"[Epoch {epoch}]")
        for step, (X1, X2) in enumerate(pbar):
            d_loss, g_loss, fake = train_on_batch(X1, X2)
            pbar.set_postfix({"g_loss": g_loss.numpy(), "d_loss": d_loss.numpy()})
            if step == 200000 // batch_size - 1:
                fake_img = fake # 末尾だと端数が出るため

[Epoch 0]: : 0it [00:02, ?it/s]


ValueError: ignored

In [0]:
for step, (X1, X2) in enumerate(pbar):
    d_loss, g_loss, fake = train_on_batch(X1, X2)
    pbar.set_postfix({"g_loss": g_loss.numpy(), "d_loss": d_loss.numpy()})
    if step == 200000 // batch_size - 1:
        fake_img = fake # 末尾だと端数が出るため

In [0]:
# お持ち帰り用
!zip -r celeba_out.zip celeba_out/*

  adding: celeba_out/epoch_0000.png (deflated 0%)
  adding: celeba_out/epoch_0001.png (deflated 0%)
  adding: celeba_out/epoch_0002.png (deflated 0%)
  adding: celeba_out/epoch_0003.png (deflated 0%)
  adding: celeba_out/epoch_0004.png (deflated 0%)
  adding: celeba_out/epoch_0005.png (deflated 0%)
  adding: celeba_out/epoch_0006.png (deflated 0%)
  adding: celeba_out/epoch_0007.png (deflated 0%)
  adding: celeba_out/epoch_0008.png (deflated 0%)
  adding: celeba_out/epoch_0009.png (deflated 0%)
  adding: celeba_out/epoch_0010.png (deflated 0%)
  adding: celeba_out/epoch_0011.png (deflated 0%)
  adding: celeba_out/epoch_0012.png (deflated 0%)
  adding: celeba_out/epoch_0013.png (deflated 0%)
  adding: celeba_out/epoch_0014.png (deflated 0%)
  adding: celeba_out/epoch_0015.png (deflated 0%)
  adding: celeba_out/epoch_0016.png (deflated 0%)
  adding: celeba_out/epoch_0017.png (deflated 0%)
  adding: celeba_out/epoch_0018.png (deflated 0%)
  adding: celeba_out/epoch_0019.png (deflated 0%)


In [0]:
from google.colab import files
files.download("celeba_out.zip")